In [10]:
import threading

def thread_job():
    print('This is an added Thread,number is %s'%threading.current_thread())

def main():
    # 加一个线程
    added_thread = threading.Thread(target=thread_job)
    # 执行这个线程
    added_thread.start()
    
    # 打印现在运行的线程数
    print(threading.active_count())
    # 打印现在运行的线程名称
    print(threading.enumerate())
    # 打印当前运行的线程名称：主线程
    print(threading.current_thread())

if __name__=='__main__':
    main()

This is an added Thread,number is <Thread(Thread-9, started 3832)>5

[<_MainThread(MainThread, started 5836)>, <Thread(Thread-4, started daemon 4408)>, <Heartbeat(Thread-5, started daemon 6540)>, <HistorySavingThread(IPythonHistorySavingThread, started 5152)>]
<_MainThread(MainThread, started 5836)>


In [11]:
import threading

def thread_job():
    print('This is an added Thread,number is %s'%threading.current_thread())

def main():
    # 加一个线程
    added_thread = threading.Thread(target=thread_job)
    # 执行这个线程
    added_thread.start()
    
if __name__=='__main__':
    main()

This is an added Thread,number is <Thread(Thread-10, started 6444)>


In [14]:
# 同时进行的线程任务，没有等到所有线程都结束就输出了 all done

import threading
import time

def thread_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1)
    print("T1 finish\n")
        
        
def main():
    # 加一个线程
    added_thread = threading.Thread(target=thread_job,name="T1")
    # 执行这个线程
    added_thread.start()
    print("all done\n")
    
if __name__=='__main__':
    main()

T1 start
all done


T1 finish



In [17]:
# 等到所有线程都结束才输出 all done

import threading
import time

def thread_job():
    print("T1 start\n")
    for i in range(10):
        time.sleep(0.1)
    print("T1 finish\n")
        
def T2_job():
    print("T2 start\n")
    print("T2 finish\n")
        
        
def main():
    # 加一个线程
    added_thread = threading.Thread(target=thread_job,name="T1")
    thread2 = threading.Thread(target=T2_job,name='T2')
    # 执行这个线程
    added_thread.start()
    thread2.start()
    added_thread.join()
    thread2.join()
    print("all done\n")
    
if __name__=='__main__':
    main()

T1 start

T2 start

T2 finish

T1 finish

all done



In [21]:
import threading
import time
from queue import Queue

def job(l,q):
    for i in range(len(l)):
        l[i] = l[i]**2
    # 多线程无法return
    # return l
    q.put(l)

def multithreading():
    q=Queue()
    threads =[]
    data = [[1,2,3],[3,4,5],[4,4,4],[5,5,5]]
    for i in range(4):
        t = threading.Thread(target=job,args=(data[i],q))
        t.start()
        threads.append(t)
    for thread in threads:
        thread.join()
    results = []
    for i in range(4):
        results.append(q.get())
    print(results)

if __name__ == '__main__':
    multithreading()

[[1, 4, 9], [9, 16, 25], [16, 16, 16], [25, 25, 25]]


In [24]:
import threading
from queue import Queue
import copy
import time

def job(l, q):
    res = sum(l)
    q.put(res)

def multithreading(l):
    q = Queue()
    threads = []
    for i in range(4):
        t = threading.Thread(target=job, args=(copy.copy(l), q), name='T%i' % i)
        t.start()
        threads.append(t)
    [t.join() for t in threads]
    total = 0
    for _ in range(4):
        total += q.get()
    print(total)

def normal(l):
    total = sum(l)
    print(total)

if __name__ == '__main__':
    l = list(range(1000000))
    s_t = time.time()
    normal(l*4)
    print('normal: ',time.time()-s_t)
    s_t = time.time()
    multithreading(l)
    print('multithreading: ', time.time()-s_t)

# 相当于扣除了输入输出的运算

1999998000000
normal:  0.29420995712280273
1999998000000
multithreading:  0.2504551410675049


In [26]:
# 打印的很乱
import threading

def job1():
    global A
    for i in range(10):
        A+= 1
        print('job1',A)

def job2():
    global A
    for i in range(10):
        A+= 10
        print('job2',A)
    
    
if __name__ == '__main__':
    A = 0
    t1 = threading.Thread(target=job1)
    t2 = threading.Thread(target=job2)
    t1.start()
    t2.start()
    t1.join()
    t2.join()

job1job2 1
job1 12
job1 13
job1 14
job1 15
job1 16
job1 17
 11
job2 27
job2 37
job2 47
job2 57
job2 67
job2 77
job2 87
job2 97
job2 107
job1 108
job1 109
job1 110


In [27]:
# 我们想先做完job1在做job2，用到了 lock

import threading

def job1():
    global A,lock
    lock.acquire()
    for i in range(10):
        A+= 1
        print('job1',A)
    lock.release()

def job2():
    global A,lock
    lock.acquire()
    for i in range(10):
        A+= 10
        print('job2',A)
    lock.release()
    
    
if __name__ == '__main__':
    lock = threading.Lock()
    A = 0
    t1 = threading.Thread(target=job1)
    t2 = threading.Thread(target=job2)
    t1.start()
    t2.start()
    t1.join()
    t2.join()


job1 1
job1 2
job1 3
job1 4
job1 5
job1 6
job1 7
job1 8
job1 9
job1 10
job2 20
job2 30
job2 40
job2 50
job2 60
job2 70
job2 80
job2 90
job2 100
job2 110
